In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import time
import os
import sys
import json
import pickle

import scipy as sp
from scipy import signal

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Conv2D, MaxPool2D, Dropout,Flatten, Conv1D, MaxPool1D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import metrics

#from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.graphics.tsaplots import plot_acf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import os, warnings, random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import keras.layers as L

from keras import Model
from keras.layers import Layer
from tensorflow.python.keras import backend as K
#from keras.backend import backend as K
#import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

from tensorflow.python.keras import optimizers as optimizers


2024-02-04 21:36:53.357806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 21:36:53.357879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 21:36:53.361491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 21:36:53.383349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 21:36:54.190862: W tensorflow/compiler/tf2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Num GPUs Available:  1


2024-02-04 21:36:58.084634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:36:58.086276: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:36:58.086630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
def encode_phase(df, name):
    phase = df[name].values.tolist()
    x = np.zeros((len(df[name]),1))
    y = np.zeros((len(df[name]),1))
    for i in range (len(phase)): 
        phi = math.radians(phase[i])
        x[i] = math.cos(phi)
        y[i] = math.sin(phi)

    name_x = name[0] + '_ph_hip_x'
    name_y = name[0] + '_ph_hip_y'

    df[name_x] = x
    df[name_y] = y

In [4]:
def encode_gait_percentage(df, name):
    percent = df[name].values.tolist()
    X = np.zeros((len(df[name]),1))
    Y = np.zeros((len(df[name]),1))
    for i in range (len(percent)):
        phi = percent[i] * 2 * math.pi / 100
        X[i] = math.cos(phi)
        Y[i] = math.sin(phi)
    df['X'] = X
    df['Y'] = Y
  
def convert_data(d_x, d_y, look_back = 10, fore_cast = 5):
    dataX = []
    dataY = []

    for i in range(look_back, len(d_x) - fore_cast):
        dataX.append(d_x[i - look_back: i])
        dataY.append(d_y[i + fore_cast,:])

    return np.array(dataX), np.array(dataY)

def get_train_data_from_df(all_data, test_ratio):
    all_data.info()
    cycle = 0
    cycle_list = []

    start = 0

    for i in range(len(all_data) - 1):
        if (all_data['perc'][i+1] == 0):
            cycle += 1
            cycle_list.append(all_data.iloc[start:i])
            start = i+1

    random.shuffle(cycle_list)
    source_table = pd.concat(cycle_list, axis=0, ignore_index=True)

    # For hip: source_table = source_table.drop(["lgrf", "rgrf", "lcop","rcop"], axis = 1)

    # all features:  "lcop", "rcop", "lgrf", "rgrf", "l_ph_hip", "r_ph_hip", "st_sw_phase_l", "st_sw_phase_r", "st_l_l", "st_l_r", "leg_len", "weight"

    drop_list = ["lgrf", "rgrf","lcop","l_ph_hip", "r_ph_hip", "st_sw_phase_l", "st_l_l"]
    source_table = source_table.drop(["lgrf", "rgrf","lcop","rcop", "st_sw_phase_l", "st_sw_phase_r", "leg_len", "weight"], axis = 1)
    source_table
    x = source_table
    x = x.drop(['perc'], axis=1)
    # scaler = MinMaxScaler()
    # x_scaled = scaler.fit_transform(x)
    # x_scaled = pd.DataFrame(x_scaled)

    encode_gait_percentage(source_table, 'perc')
    y = source_table[["X","Y"]]
    # data_x = x_scaled
    data_x=x.values
    x.info()
    data_y = y.values.reshape(-1,2)

    X_train, X_test,y_train, y_test = train_test_split(data_x, data_y ,
                            test_size=0.25,
                            shuffle=False)
    look_back = 10
    fore_cast = 1

    train_x, train_y = convert_data(X_train, y_train, look_back, fore_cast)
    validation_x, validation_y = convert_data(X_test, y_test, look_back, fore_cast)

    return train_x, train_y, validation_x, validation_y


import random
def get_data_frames_from_files(path, file_names, subject_dict, subjects):
    file_list=[]
    for i in range (len(file_names)):

        subject = file_names[i].split('_')[0]
        if not subject in subjects:
            continue
        else:
            print(file_names[i])
        leg_len = subject_dict[subject][0]
        weight = subject_dict[subject][1]
        tmp=pd.read_excel(path+ file_names[i], sheet_name='Sheet1')
        perc_column = tmp['perc']
        tmp = tmp.drop(columns=['perc'])
        scaler = MinMaxScaler()

    # Normalize each column separately
        normalized_data = scaler.fit_transform(tmp)
        column_names = tmp.columns

        normalized_df = pd.DataFrame(normalized_data, columns=column_names)
        tmp['l_ph_hip']=tmp['l_ph_hip']/300
        tmp['r_ph_hip']=tmp['r_ph_hip']/300
        tmp['l_ph_fo']=tmp['l_ph_fo']/300
        tmp['r_ph_fo']=tmp['r_ph_fo']/300
        tmp['lcop']= tmp['lcop']*1000
        tmp['rcop']=tmp['rcop']*1000
        tmp['strike_frame']=tmp['strike_frame']/400
        tmp['st_sw_phase']=tmp['st_sw_phase']/200
        normalized_df =tmp
        normalized_df['leg_len']=leg_len
        normalized_df['weight']=weight
        normalized_df['perc']= perc_column
        # normalized_df.insert(tmp.columns.get_loc('col1'), 'perc', perc_column)

        file_list.append(normalized_df)

    random.shuffle(file_list)

    all_data = pd.concat(file_list, axis=0, ignore_index=True)
    return all_data


seq_len=10
n_features=11
#train_x=np.zeros((10,10,11))
#encoder_model().summary()
#decoder_model().summary()

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        curLR = optimizer.lr
        return curLR
    return lr

def train_mlp_model(train_x, train_y, look_back):
    epochs = 30
    batch_size = 32                                                                
    learning_rate = 1e-2

    # learning rate scheduler
    use_lr_scheduler = True
    lr_factor = 0.5
    lr_patience = 8
    lr_threshold = 0.01
    lr_min_rate = 1e-6

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    mlp_model = Sequential()
    mlp_model.add(LSTM(64, input_shape=(look_back, train_x.shape[2]), return_sequences = True, dropout=0.1))
    #model.add(LSTM(64, return_sequences = True))
    mlp_model.add(LSTM(32))
    mlp_model.add(Dense(2))

    print(mlp_model.summary())

    monitor = EarlyStopping(monitor='loss', min_delta=1e-5, patience=5, verbose=1, mode='auto')

    #optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    mlp_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy','mae','mse'])
    history = mlp_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, callbacks=monitor,verbose=1)
   
    return mlp_model

def test_model_get_results(mlp_model, validation_x, validation_y, display_flag, tag, file):
    print("validation_x shape",validation_x.shape)
    testPredict = mlp_model.predict(validation_x)
    print(validation_y.shape)
    print(testPredict.shape)
    testScore = np.sqrt(mean_squared_error(validation_y, testPredict))
    # print(testScore)
    # print('Test Score: %.2f RMSE' % (testScore))
    pred = np.zeros((len(testPredict),1))

    for iter in range(len(testPredict)):
        x = testPredict[iter][0]
        y = testPredict[iter][1]
        pred[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi)) * (100 / (2*math.pi))

    actual = np.zeros((len(validation_y),1))

    for iter in range(len(validation_y)):
        x =validation_y[iter][0]
        y =validation_y[iter][1]
        # st_frame = validation_x[0,9,7]

        actual[iter] = ((math.atan2(y,x) + 2*math.pi) % (2*math.pi)) * (100 / (2*math.pi))
        # if st_frame>100 and pred[iter]<15:
        #     pred[iter]=100-pred[iter]
        # if st_frame<90 and pred[iter]>85:
        #     pred[iter]=100-pred[iter]


    cor_actual=[]
    cor_pred=[]
    prec_list=[]

    for i in range(5):
        correct = 0
        for iter in range(len(actual)):
            # if (actual[iter]>98) or(actual[iter]<3):
            #     correct+=1
            #     continue
            if (abs(actual[iter] - pred[iter]) <= (i+1)):
                correct+=1
            cor_pred.append(pred[iter])
            cor_actual.append(actual[iter])
        prec=correct * 100/len(actual)
        print("Precision ", i+1, ": ", prec)
        # file.write(str(prec))
        # file.write("\n")
        prec_list.append(prec)

    rmse = 0
    length = len(actual)
    for i in range(len(actual)):
        if abs (pred[i] - actual[i]) >=90:
            length -= 1
        else:
            rmse = rmse + pow(pred[i] - actual[i], 2)
    rmse = rmse / length
    rmse = math.sqrt(rmse)

    if os.path.exists(result_pkl_file):
        with open(result_pkl_file,'rb') as pkl_file:
            res_dict=pickle.load(pkl_file)
    else:
        res_dict={}
    result={}
    result['rmse']=rmse
    result['prec_list']=prec_list
    res_dict[tag]=result
    with open(result_pkl_file, 'wb')as pkl_file:
        pickle.dump(res_dict, pkl_file)


    print(rmse)
    # file.write("rmse "+str(rmse))
    # file.write("\n")
    if display_flag:
        plt.scatter(cor_actual, cor_pred, facecolors='none', edgecolors='crimson',alpha=0.4)
        p1 = max(max(cor_pred), max(cor_actual))
        p2 = min(min(cor_pred), min(cor_actual))

        ci = 0.1 * np.std([p1,p2]) / np.mean([p1,p2])

        plt.plot([p1, p2], [p1, p2], 'b-', linewidth =3)
        plt.title('Actual vs Prediction')
        plt.savefig(result_path+tag+"__res.png")

    return prec_list, rmse


In [5]:
file_names = ['JL_I_0_new_.xlsx', 'JL_I_2_new_.xlsx','JL_I_3_new_.xlsx','JL_I_5_new_.xlsx','JL_I_4_new_.xlsx',
              'JS_I_1_new_.xlsx', 'JS_I_2_new_.xlsx','JS_I_3_new_.xlsx','JS_I_5_new_.xlsx','JS_I_4_new_.xlsx',
              'AK_I_0_new_.xlsx', 'AK_I_2_new_.xlsx','AK_I_3_new_.xlsx','AK_I_5_new_.xlsx','AK_I_4_new_.xlsx',
              'VN_I_0_new_.xlsx', 'VN_I_2_new_.xlsx','VN_I_3_new_.xlsx','VN_I_5_new_.xlsx','VN_I_4_new_.xlsx',
              'VP_I_0_new_.xlsx', 'VP_I_2_new_.xlsx','VP_I_3_new_.xlsx','VP_I_5_new_.xlsx','VP_I_4_new_.xlsx',
             'SOE_I_0_new_.xlsx', 'SOE_I_2_new_.xlsx','SOE_I_3_new_.xlsx','SOE_I_5_new_.xlsx', 'SOE_I_4_new_.xlsx', 'SD_I_3_new_.xlsx', 'SD_I_4_new_.xlsx','SD_I_5_new_.xlsx',
             'SD_I_1_new_.xlsx','SD_I_2_new_.xlsx','TH_I_0_new_.xlsx', 'TH_I_2_new_.xlsx', 'TH_I_3_new_.xlsx','TH_I_4_new_.xlsx', 'TH_I_5_new_.xlsx'
             ,'PK_I_0_new_.xlsx', 'PK_I_2_new_.xlsx', 'PK_I_3_new_.xlsx','PK_I_5_new_.xlsx',
              'SKS_0_I_new_.xlsx', 'SKS_2_I_new_.xlsx','SKS_3_I_new_.xlsx','SKS_4_I_new_.xlsx','SKS_5_I_new_.xlsx',
            'PH_I_0_new_.xlsx',  'PH_I_2_new_.xlsx',  'PH_I_3_new_.xlsx',  'PH_I_4_new_.xlsx',  'PH_I_5_new_.xlsx',
            'YC_I_0_new_.xlsx',  'YC_I_2_new_.xlsx',  'YC_I_3_new_.xlsx',  'YC_I_4_new_.xlsx',  'YC_I_5_new_.xlsx'
              ]
subject_dict = {'VN':[0.90,0.63],'AK':[0.80,0.57],'JS':[0.89,0.64],'JL':[0.79,0.63],'SKS':[0.83, 0.58],'VP':[0.93, 0.77],'SOE':[0.90, 0.83],
                'SD':[0.83, 0.70], 'TH':[0.66, 0.52], 'PK':[0.90, 0.88], 'PH':[0.95,0.80], 'YC':[0.82,0.79]}
subject_names = ['VP','AK', 'SD','PH', 'YC','JL','JS', 'PK', 'SOE', 'TH','SKS', 'VN']#,'VN','AK' 'SOE'
sub_comb_list=[]
test_sub_list=[]
acc_list=[]
rmse_list=[]
test_acc_list=[]
test_rmse_list=[]


#path="/home/vtp/Gait_Phase_Prediction/Subject_data/Final_files/"
#result_path = "/home/vtp/Gait_Phase_Prediction/Results/final/"

path = "/home/hikikomori/Gait_Phase_Prediction/Data/Final Data/"
result_path = "/home/hikikomori/Gait_Phase_Prediction/Results/final/"

result_pkl_file=result_path+'results_ver_cnn_2.pkl'
pkl_file=path+"all_sub_cnn_2.pkl"
# pkl_file=path+"good_sub_data.pkl"

for sub in subject_names:

  test_sub_list.append(sub)
  tmp=subject_names.copy()
  tmp.remove(sub)
  sub_comb_list.append(tmp)

df_dict={}

if os.path.exists(pkl_file):
    # File is already in pickle format, read to dict
    with open(pkl_file, 'rb') as file:
        df_dict = pickle.load(file)
    
else:
    # File is not in pickle format/ does not exist, convert and save it as a pickle file

    for file_name in file_names:
        subject = file_name.split('_')[0]
        if subject not in subject_names:
            continue
        leg_len = subject_dict[subject][0]
        weight = subject_dict[subject][1]
        print("Reading file :", file_name)
        tmp=pd.read_excel(path+ file_name, sheet_name='Sheet1')

        #### Add r_ph_hip_l and r
        #encode_phase(tmp, 'r_ph_hip')
    
        perc_column = tmp['perc']
        st_sw_col_l = tmp['st_sw_phase_l']
        st_sw_col_r= tmp['st_sw_phase_r']
        lhip_col = tmp['lhip_ang']
        rhip_col = tmp['rhip_ang']
        lhip_df=tmp['l_ph_hip']
        rhip_df=tmp['r_ph_hip']
        #r_hip_ph_x = tmp['r_ph_hip_x']
        #r_hip_ph_y = tmp['r_ph_hip_y']

        
        st_l_l_col = tmp[['st_l_l']]
        st_l_r_col = tmp[['st_l_r']]

        tmp = tmp.drop(columns=['perc', 'st_sw_phase_l', 'st_sw_phase_r', 'lhip_ang', 'rhip_ang','l_ph_hip','r_ph_hip', 'st_l_l', 'st_l_r'])#, 'r_ph_hip_x', 'r_ph_hip_y'])

        scaler = MinMaxScaler()
        tmp['l_ph_hip'] = lhip_df
        tmp['r_ph_hip'] = rhip_df
        tmp['st_sw_phase_l'] = st_sw_col_l
        tmp['st_sw_phase_r'] = st_sw_col_l
        #tmp['r_ph_hip_x'] = r_hip_ph_x
        #tmp['r_ph_hip_y'] = r_hip_ph_y
        tmp['st_l_l'] = scaler.fit_transform( st_l_l_col)
        tmp['st_l_r'] = scaler.fit_transform( st_l_r_col)

        column_names = tmp.columns


        tmp['l_ph_hip']=tmp['l_ph_hip']/300
        tmp['r_ph_hip']=tmp['r_ph_hip']/300
        #tmp['r_ph_hip_x'] = tmp['r_ph_hip_x'] / 300
        #tmp['r_ph_hip_y'] = tmp['r_ph_hip_y'] / 300
        # tmp['l_ph_fo']=tmp['l_ph_fo']/300
        # tmp['r_ph_fo']=tmp['r_ph_fo']/300
        tmp['lcop']= tmp['lcop']*1000
        tmp['rcop']=tmp['rcop']*1000
        tmp['st_sw_phase_l']=tmp['st_sw_phase_l']/200
        tmp['st_sw_phase_r']=tmp['st_sw_phase_r']/200

        normalized_df =tmp

        normalized_df['leg_len']=leg_len
        normalized_df['weight']=weight
        normalized_df['perc']= perc_column

        df_dict[subject] = normalized_df
    with open(pkl_file,'wb') as pickle_file:
        pickle.dump(df_dict, pickle_file)

#sub_cnt_file='sub_cnt.pkl'

with open(result_path+"all_results_cnn_2.txt","w") as file:  
    for sub_iter, sub_comb in enumerate(sub_comb_list):
    #for a_ in range(len(subject_names)):
        #sub_file=open(sub_cnt_file,'rb')
        #cnt=pickle.load(sub_file)
        #sub_file.close()
        #if cnt==len(subject_names):
        #    print("Done with all subjects !!")
        #    break
        #sub_comb=sub_comb_list[cnt]
        #sub_iter=cnt
        print("Subject combination :",sub_comb)
        sub_tag=test_sub_list[sub_iter]
        # all_data=get_data_frames_from_files(path, file_names, subject_dict, sub_comb)
        file_list=[]
        for sub in sub_comb:
          tmp_df=df_dict[sub]
          file_list.append(tmp_df)
        random.shuffle(file_list)
        
        all_data = pd.concat(file_list, axis=0, ignore_index=True)
        print("All Data Info: ")
        all_data.info()
        train_x, train_y, validation_x, validation_y = get_train_data_from_df(all_data, 0.25)
        print("Nan in train x at indices:",np.argwhere(np.isnan(train_x)))
        print("Nan in train y at indices:",np.argwhere(np.isnan(train_y)))

        print(train_x.shape)
        print(validation_x.shape)
        print("SSS")
        seq_len = train_x.shape[1]
        n_features = train_x.shape[2]
     
        mlp_model=[]
        print("OKAY till here 1")

        look_back = 10

        mlp_model = train_mlp_model(train_x, train_y, look_back)
        file.write("Training Result :")
        file.write("\n")
        acc, rmse=test_model_get_results(mlp_model, validation_x, validation_y, False, sub_tag, file)
        acc_list.append(acc)
        rmse_list.append(rmse)
        print("testing on :", test_sub_list[sub_iter])
        file.write("Testing  Result :")
        file.write(test_sub_list[sub_iter])
        file.write("\n")
        # all_data=get_data_frames_from_files(path, file_names, subject_dict, [test_sub_list[sub_iter]])
        all_data = df_dict[test_sub_list[sub_iter]]
        test_x, test_y, validation_x, validation_y = get_train_data_from_df(all_data, 0.25)
        acc, rmse=test_model_get_results(mlp_model, test_x, test_y, True, sub_tag, file)
        # tf.keras.backend.clear_session()

        test_acc_list.append(acc)
        test_rmse_list.append(rmse)
        # break
        # file.write(acc)
        file.write("\n")
        #cnt=cnt+1
        #sub_file=open(sub_cnt_file,'wb')
        #pickle.dump(cnt, sub_file)
        #sub_file.close()
        # if sub_iter>=5:
        # break
    file.close()


Subject combination : ['AK', 'SD', 'PH', 'YC', 'JL', 'JS', 'PK', 'SOE', 'TH', 'SKS', 'VN']
All Data Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572285 entries, 0 to 572284
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lcop           572285 non-null  float64
 1   rcop           572285 non-null  float64
 2   lgrf           572285 non-null  float64
 3   rgrf           572285 non-null  float64
 4   l_ph_hip       572285 non-null  float64
 5   r_ph_hip       572285 non-null  float64
 6   st_sw_phase_l  572285 non-null  float64
 7   st_sw_phase_r  572285 non-null  float64
 8   st_l_l         572285 non-null  float64
 9   st_l_r         572285 non-null  float64
 10  leg_len        572285 non-null  float64
 11  weight         572285 non-null  float64
 12  perc           572285 non-null  float64
dtypes: float64(13)
memory usage: 56.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572285 entries,

2024-02-04 21:37:01.905553: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:37:01.906035: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:37:01.906373: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:37:06.789790: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-04 21:37:06.789837: I tensorflow/core/common_runtime/gpu/gpu

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)            │     17,664 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)                │     12,416 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 2)                 │         66 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 30,146 (117.76 KB)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 554s 42ms/step - accuracy: 0.9474 - loss: 0.0376 - mae: 0.1218 - mse: 0.0376
Epoch 2/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 555s 42ms/step - accuracy: 0.9679 - loss: 0.0138 - mae: 0.0760 - mse: 0.0138
Epoch 3/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 576s 43ms/step - accuracy: 0.9693 - loss: 0.0133 - mae: 0.0749 - mse: 0.0133
Epoch 4/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 542s 41ms/step - accuracy: 0.9679 - loss: 0.0133 - mae: 0.0748 - mse: 0.0133
Epoch 5/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 550s 42ms/step - accuracy: 0.9684 - loss: 0.0134 - mae: 0.0750 - mse: 0.0134
Epoch 6/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 557s 42ms/step - accuracy: 0.9682 - loss: 0.0135 - mae: 0.0751 - mse: 0.0135
Epoch 7/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 556s 42ms/step - accuracy: 0.9686 - loss: 0.0135 - mae: 0.0750 - mse: 0.0135
Epoch 8/30
13234/13234 ━━━━━━━━━━━━━━━━━━━━ 556s 42ms/step - accuracy: 0.9685 - loss: 0.0134 - mae: 0.0748 - mse: 0.0134
Epoch 9/30
13234/13234 ━━━━

/tmp/ipykernel_32075/128050465.py:217: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


7.621203422044127
testing on : TH
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59620 entries, 0 to 59619
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lcop           59620 non-null  float64
 1   rcop           59620 non-null  float64
 2   lgrf           59620 non-null  float64
 3   rgrf           59620 non-null  float64
 4   l_ph_hip       59620 non-null  float64
 5   r_ph_hip       59620 non-null  float64
 6   st_sw_phase_l  59620 non-null  float64
 7   st_sw_phase_r  59620 non-null  float64
 8   st_l_l         59620 non-null  float64
 9   st_l_r         59620 non-null  float64
 10  leg_len        59620 non-null  float64
 11  weight         59620 non-null  float64
 12  perc           59620 non-null  float64
dtypes: float64(13)
memory usage: 5.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59133 entries, 0 to 59132
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ----

/tmp/ipykernel_32075/128050465.py:217: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rmse = math.sqrt(rmse)


In [ ]:
personInfo = pickle.load(open(result_pkl_file, "rb"))
print(personInfo)

{'VP': {'rmse': 5.8427562468088565, 'prec_list': [10.85915043117215, 27.339508144362824, 49.47546863867528, 71.17902857283246, 84.94951232095914]}, 'AK': {'rmse': 1.9297164656753376, 'prec_list': [41.296591869308045, 70.10139892967797, 86.75945920570838, 94.16721434607079, 96.79372828842362]}, 'SD': {'rmse': 2.3068117510901263, 'prec_list': [41.18044604640685, 64.56972291056545, 77.80750168956972, 86.77911691822483, 92.21389952692047]}, 'PH': {'rmse': 1.74826790337621, 'prec_list': [46.70817296807788, 76.80552411138783, 89.44532488114105, 94.12044374009508, 96.29612859406838]}, 'YC': {'rmse': 10.786383597340192, 'prec_list': [9.760264685144003, 21.009383305309974, 34.373813527146496, 46.88669523241308, 57.59613820035798]}, 'JL': {'rmse': 2.39928991842349, 'prec_list': [39.224727791991434, 63.96441958707681, 79.97858035342416, 88.54048908193015, 93.07728922472779]}, 'JS': {'rmse': 2.181442599964781, 'prec_list': [34.77336469736723, 63.978105491721706, 82.49796435356917, 91.8234868361530